In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

### Generate synthetic data

In [17]:
from synthetics import generate_single_task_unipolar, gaussian_bags_of_words, vocab1k

N = 1000
M = 30
K = 2
NUM_SPLITS = 3

Ds = [] # data
Xs = [] # features
Ls = [] # noisy labels
Ys = [] # true labels
for _ in range(NUM_SPLITS):
    L, Y, _ = generate_single_task_unipolar(
        N, M, k=K, acc=[0.6, 0.9], rec=[0.1, 0.2], 
        class_balance=[0.3, 0.7], lf_balance=None, seed=1)
    
    X, D = gaussian_bags_of_words(Y, vocab1k)
    
    Ls.append(L)
    Ys.append(Y)
    Ds.append(D)
    Xs.append(X)

### Apply LabelModel

In [18]:
from metal.label_model import LabelModel

lm = LabelModel(seed=2)
lm.train(Ls[0], n_epochs=50)
Y_p = lm.score(Ls[1], Ys[1])

Reaffirming seed=2
Reaffirming n_epochs=50
[Epoch 0] Loss: 0.180101
[Epoch 10] Loss: 0.157589
[Epoch 20] Loss: 0.119086
[Epoch 30] Loss: 0.086237
[Epoch 40] Loss: 0.070783
[Epoch 49] Loss: 0.067910
Finished Training
Accuracy: 0.717


In [19]:
Y_p = lm.predict(Ls[1])

### Metrics

Calculate metrics in one of two ways.

1. Use metric_score() and pass the metric name
3. The the specific metric's function (e.g., accuracy_score())

In [20]:
from metal.metrics import metric_score, accuracy_score

metric_score(Ys[1], Y_p, 'accuracy')
accuracy_score(Ys[1], Y_p)

0.717

0.717

Built-in metrics include:

In [21]:
metric_score(Ys[1], Y_p, 'accuracy')
metric_score(Ys[1], Y_p, 'coverage')
metric_score(Ys[1], Y_p, 'precision')
metric_score(Ys[1], Y_p, 'recall')
metric_score(Ys[1], Y_p, 'f1')
metric_score(Ys[1], Y_p, 'fbeta', beta=1.0)

0.717

1.0

0.5156537753222836

0.9333333333333333

0.6642941874258601

0.6642941874258601

### Confusion matrix

In [22]:
from metal.analysis import confusion_matrix
confusion_matrix(Y_p, Ys[1])

array([[280, 263],
       [ 20, 437]])

In [23]:
confusion_matrix(Y_p, Ys[1], normalize=True)

array([[0.28 , 0.263],
       [0.02 , 0.437]])

In [24]:
mat = confusion_matrix(Y_p, Ys[1], pretty=True)

        y=1    y=2   
 l=1    280    263   
 l=2    20     437   


### Error analysis

In [10]:
from metal.analysis import error_buckets

buckets = error_buckets(Y_p, Ys[1], Ds[1])
tp = buckets[1,1]

In [11]:
tp[:3]

['him fight mr within hope economy visit parent piece problem discuss camera join factor day should night personal music week sell catch manager base them chance job later away legal period left treatment successful control maintain if fail throw',
 'professional operation arm response or organization city affect agency protect case simple defense shoulder such process patient property on large gun somebody side employee story their well court',
 'animal sell think inside similar manage than these might ten or lose machine in hear land customer audience nearly attack go college capital weight social lead answer recent wall us individual response serious he team']